# T-Maze Inference for Planning with EFE

This notebook reports the EFE for all policies for a variety of scenarios, and is used to generate Fig. 7 (Sec. 6.2).

In [ ]:
using LinearAlgebra
using Plots

# T-maze layout
# [2| |3]
#   | |
#   |1|
#   |4|

# EFE Algorithm

In [ ]:
function evaluateEFE(A, B, C, D)
    # Construct priors
    # C_t = [C, C]
    C_t = [ones(16)/16, C]
    D_t_min = D
    
    # Evaluate all policies
    Q = zeros(4,4)
    for i in 1:4 # First move
        x_t_hat = B[i]*D_t_min # Expected state
        y_t_hat = A*x_t_hat # Expected outcome

        # We follow Eq. D.2 in da Costa (2021) "Active inference on discrete state-spaces: a synthesis",
        # which algebraically is the same as Friston (2015), but reads better.
        predicted_uncertainty_t = diag(A' * log.(A.+eps()))' * x_t_hat # Friston (for reference): ones(16)' * (A.*log.(A.+eps())) * x_t_hat
        predicted_divergence_t = transpose( log.(y_t_hat.+eps()) - log.(C_t[1].+eps()) ) * y_t_hat
        Q_t = predicted_uncertainty_t - predicted_divergence_t

        for j in 1:4 # Second move
            x_t_plus_hat = B[j]*x_t_hat # Expected state
            y_t_plus_hat = A*x_t_hat # Expected outcome

            predicted_uncertainty_t_plus = diag(A' * log.(A.+eps()))' * x_t_plus_hat
            predicted_divergence_t_plus = transpose( log.(y_t_plus_hat.+eps()) - log.(C_t[2].+eps()) ) * y_t_plus_hat
            Q_t_plus = predicted_uncertainty_t_plus - predicted_divergence_t_plus

            Q[i, j] = Q_t + Q_t_plus
        end
    end

    return -Q./log(2) # Return expected free energy per policy in bits
end
;

# EFE Results

In [ ]:
# Reward probability and utility, uncomment to select scenario
α = 0.9; c = 2.0
# α = 0.5; c = 2.0
# α = 0.9; c = 0.0
# α = 1.0; c = 2.0

include("environment.jl")
include("agent.jl")
include("helpers.jl")

(A, B, C, D) = constructABCD(α, c)
;

In [ ]:
EFE = evaluateEFE(A, B, C, D)
plotResults(EFE, clim=(7.0,12.0), dpi=300, highlight=minimum)
savefig("figures/G_c_$(c)_a_$(α).png")